In [4]:
import nltk

nltk.download('nps_chat')
posts = nltk.corpus.nps_chat.xml_posts()[:10000]

def dialogue_act_features(post):
    features = {}
    for word in nltk.word_tokenize(post):
        features[f'contains({word.lower()})'] = True
    return features

def generate_binary_feature(label):
    return label in ['whQuestion', 'yAnswer','ynQuestion']

featuresets = [(dialogue_act_features(post.text), generate_binary_feature(post.get('class'))) for post in posts]

# 10% of the total data
size = int(len(featuresets) * 0.2)

# first 10% for test_set to check the accuracy, and rest 90% after the first 10% for training
train_set, test_set = featuresets[size:], featuresets[:size]

# get the classifier from the training set
classifier = nltk.NaiveBayesClassifier.train(train_set)

# to check the accuracy
print("Accuracy:", nltk.classify.accuracy(classifier, test_set))


[nltk_data] Downloading package nps_chat to
[nltk_data]     /Users/seungwonlim/nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!


Accuracy: 0.852


In [7]:
# Now, classify a new sentence
new_sentence = "I don't really understand why"
new_features = dialogue_act_features(new_sentence)
predicted_label = classifier.classify(new_features)

print("Predicted label:", predicted_label)

Predicted label: True


In [21]:
import nltk

# Ensure required NLTK data is downloaded
nltk.download('nps_chat')
nltk.download('punkt')

# Load the NPS chat dataset
posts = nltk.corpus.nps_chat.xml_posts()[:100000]

# Filter out the posts that are classified as questions
question_posts = [post for post in posts if post.get('class') in ['whQuestion', 'ynQuestion']]

# Print the total number of question posts
print(f"Total question posts: {len(question_posts)}")

# Optionally, view some of the question posts
for post in question_posts[:100]:
    print(post.text)


[nltk_data] Downloading package nps_chat to
[nltk_data]     /Users/seungwonlim/nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/seungwonlim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Total question posts: 1083
whats everyone up to?
any ladis wanna chat? 29 m
r u serious
don't you have a sharpie?
Any ladies wanna chat with 24/m
Anyone from Tennessee in here?
is 10-19-20sUser68 back yet
Any women from Nashville in here?
are you a male?
hey any guys with cams wanna play?
any guyz wanna chat 
where did everyone gooo?
what did you but on e-bay
can't sleep huh?
well 10-19-20sUser68 what did u buy
how you doin 10-19-20sUser139
how many kts
any single white females?
off off?
really?
Just fine thanks, how are you?
finger?
any girls wanna chat with 24/m
yes 10-19-20sUser121??
forwads?
22/m/wa any ladies want to chat
can i talk to him!!
who is he 10-19-20sUser6?
who honey??
your fiance, do you know his handle?
On what kinda pervs your lookin for
what, 10-19-20sUser92.
handle?
lol 10-19-20sUser44 how do u know that
what?
jerkettes?? lmao
CO?
wtf
girl jerks??
ur annoying
what'd I miss?
why not?? girls are nice honey!!!
huh?
What?
aw 10-19-20sUser115 whys that
Your phone sexs is

In [17]:
posts[5].text

'NICK :10-19-20sUser7'

In [19]:
posts[0].get('class')

'Statement'